In [62]:
# Imports for scraping
from bs4 import BeautifulSoup
import requests
import re
from collections import defaultdict



In [94]:
def scrape_page(url):
    """
    Scrapes and process text from a webpage, focusing on <p> tags.

    Parameters:
    url (str): The URL of the webpage to scrape.

    Returns:
    str: The cleaned-up text extracted from the web page, or an empty string if an issue occurs during scraping.
    """

    # returns class attributes of an HTML element or "NOCLASS" if it doesn't have any.
    def get_class(p):
        return ''.join(p['class']) if p.has_attr('class') else "&&NOCLASS&&"

    p_dict = defaultdict(list)

    try:
        response = requests.get(url)

        # if we've been redirected, return.
        if response.url != url:
            return ""

        soup = BeautifulSoup(response.text, 'html.parser')
        # finds all <p> tags
        paragraphs = soup.find_all('p')

        for p in paragraphs:
            # Add text to dict if there are more than 5 words in it.
            if len(p.text.split(' ')) > 5:
                p_dict[get_class(p)].append(p.text)

    except Exception as e:
        return ""

    # Find the class with the longest list of paragraphs
    longest_key = max(p_dict, key=lambda k: len(p_dict[k]))

    # Combine and clean up text
    full_text = re.sub(r'\s+', ' ', ' '.join(p_dict[longest_key])).strip()

    return full_text


Note: The writer of this review watched The Father on a digital screener from home. Before making the decision to see it—or any other film—in a movie theater, please consider the health risks involved. Here’s an interview on the matter with scientific experts. Anthony Hopkins has reached the point in his career where his very presence sparks an image in the mind: the instant impression of sophistication. This veteran of stage and screen classes up projects highbrow and low- with his playful urbanity; when he appears in, say, Thor or a Transformers movie, it’s at least in part for the instant Shakespearean associations created by his plummy accent or a poise that might well be muscle memory after so many plum parts from the Bard’s canon. But The Father, Hopkins’ new movie, does something rather disquieting with his star power and our assumed familiarity with it. To play a man who’s begun to lose his mental faculties, Hopkins methodically strips away every quality we’ve come to expect fr